    Install modules

In [ ]:
import geopandas as gpd #gpd for creating geodatabases, tables with geometry data
import matplotlib.pyplot as plt 
import pandas as pd
import shapely
import cartopy.crs as ccrs

from cartopy.feature import ShapelyFeature
from shapely.geometry import Point

Load crime data data 

In [ ]:
crime = gpd.read_file('crimeDataEdit3.csv') #crime data csv
crime.crs = 'epsg:27700'
crime.head 

Load parish data

In [ ]:
parishes = gpd.read_file('parishBound.csv') #csv with wkt for parish boundaries
parishes.crs = 'epsg:27700'
parishes.head

Convert dataframe to geodataframe with geometry values. MAY NOT BE NEEDED

In [ ]:
#crime = geopandas.GeoDataFrame(
    #df, geometry=geopandas.points_from_xy(df.X, df.Y))
#crime.crs = 'epsg:27700'
#crime.head

Test gdf have same crs

In [ ]:
print(crime.crs == parishes.crs)

Display gdf layer

In [ ]:
f, ax = plt.subplots(1, figsize=(12, 12))
ax = parishes.plot(ax=ax, alpha=0.8) #adjust alpha value to change colour intensity
#ax.set_axis_off()  #uncomment this line to remove national grid numbers
f.suptitle('Suffolk Parishes')
plt.show() 

Load a single parish using row number

In [ ]:
parishes.loc[0, 'geometry'] #adjust number to select parish

Plot data to scattergraph showing grid references. 

In [ ]:
crime.plot(figsize=(12,10))

Clip the crime gdf data to the parish gdf so it only shows crimes taking place in Suffolk Parishes

In [ ]:
clipped = []
for parish in parishes['Parish'].unique():
    tmp_clip = gpd.clip(crime, parishes[parishes['Parish'] == parish])
    clipped.append(tmp_clip)

clipped_gdf = gpd.GeoDataFrame(pd.concat(clipped))
clip_total = clipped_gdf.count()

clip_total

Rename clipped_gdf to crimes and set CRS to 27700. Display clipped crime scattergraph

In [ ]:
crimes = clipped_gdf
crimes.crs = 'epsg:27700'
crimes.plot(figsize=(12,10));

Plot clipped crime points over parish map

In [ ]:
base = parishes.plot(figsize = (20,18), color='white', edgecolor='black')

crimes.plot(ax=base, marker='d', color='blue', markersize=5)



Join the two tables, crimes and parish

In [ ]:
join = gpd.sjoin(parishes, crimes, how='inner', lsuffix='left', rsuffix='right') # perform the spatial join
join # show the joined table

Select crime type by parish.

In [ ]:
crime_stats = join.groupby(['Parish', 'Crime_type']).count()
print(crime_stats.loc['Sproughton CP']) #Edit to select parish of interest

Display the overall number of crimes per parish

In [ ]:
print(join.groupby(['Parish'])['Crime_type'].count())

Generate Choropleth map of crimes report by parish

In [ ]:
polygons = parishes
polygon_id_field = 'OBJECTID'
points = crimes
points.crs = 'epsg:27700'

sjoin = gpd.sjoin(parishes, crimes, how='left', predicate='contains')
count = sjoin.groupby(polygon_id_field)[polygon_id_field].count()
count.name='pointcount'
polygons = pd.merge(left=polygons, right=count, left_on=polygon_id_field, right_index=True)

fig, ax = plt.subplots(figsize = (20,18))
polygons.plot(column = 'pointcount', cmap = 'Spectral_r', ax=ax, legend=True,
              legend_kwds={'label':'Number of crimes reported'})
polygons.geometry.boundary.plot(color=None, edgecolor='k',linewidth = 1, ax=ax)

Reproduce Parish map, clipped crimes map, and choropleth map.